In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''

from openai import OpenAI
client = OpenAI()

Load dataset

In [13]:
import pandas as pd
df = pd.read_csv('../data/adobe_style_campaign_data.csv')

metrics = (
    df.groupby("campaign_name")
    .agg({
        "step1_landing":"sum",
        "step2_product_view":"sum",
        "step3_add_to_cart":"sum",
        "step4_checkout_start":"sum",
        "step5_purchase":"sum"
    }).reset_index()
)

metrics['landing_to_view_rate'] = metrics['step2_product_view']/metrics['step1_landing']
metrics['view_to_add_rate'] = metrics['step3_add_to_cart']/metrics['step2_product_view']
metrics['add_to_checkout_rate'] = metrics['step4_checkout_start']/metrics['step3_add_to_cart']
metrics['checkout_to_purchase_rate'] = metrics['step5_purchase']/metrics['step4_checkout_start']
metrics['landing_to_purchase_rate'] = metrics['step5_purchase']/metrics['step1_landing']
metrics.head()


,campaign_name,step1_landing,step2_product_view,step3_add_to_cart,step4_checkout_start,step5_purchase,landing_to_view_rate,view_to_add_rate,add_to_checkout_rate,checkout_to_purchase_rate,landing_to_purchase_rate
0,Campaign 1,1548584,1002506,495834,344774,266259,0.647369,0.494595,0.695342,0.772271,0.171937
1,Campaign 10,1768951,1146349,572798,402392,311139,0.648039,0.499672,0.702502,0.773224,0.175889
2,Campaign 2,1574601,1019944,509995,357840,275353,0.647748,0.500023,0.701654,0.769486,0.174872
3,Campaign 3,1567483,1023798,513059,361595,278841,0.653148,0.501133,0.704782,0.771142,0.177891
4,Campaign 4,1502271,970346,484262,338209,261574,0.645919,0.499061,0.698401,0.773409,0.174119


Build prompt template

In [ ]:
# each campaign 

def build_funnel_prompt(metrics):
    return f"""
You are an analytics assistant to a marketing team. Given traffic to the journey and the ratios between each steps in the funnel,
discuss trends, funnel issues, anomalies and recommendations for optimisation. Max 3-4 sentences.

Use the data below:
{metrics}
"""

In [37]:
# overall summary prompt 
import json

def build_summary_prompt(metrics):
    metrics_dict = metrics.to_dict(orient="records")
    metrics_json = json.dumps(metrics_dict, indent = 2)
    return f"""

You are an analytics assistant to a marketing team. Given traffic to the journey and the ratios between each steps in the funnel.

1. Summarise overall patterns across campaigns. 
2. Highlight which stages of the funnel are generally weakest. 
3. Call out any standout high and low performing campaigns.
4. Give 2-3 recommendations to improve the performance 

Use the data below:
{metrics_json}

"""

Call OpenAI LLM & apply

In [ ]:
# each campaign 

def generate_insight(metrics_row):
    prompt = build_funnel_prompt(metrics_row)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = [{"role":"user", "content": prompt}]
    )
    return response.choices[0].message.content

In [30]:
metrics['llm_insight'] = metrics.apply(lambda row: generate_insight(row.to_dict()), axis = 1)
print(metrics.loc[0,"llm_insight"])

The data for Campaign 1 reveals a significant drop-off at each step in the funnel, particularly from product views to add to cart (50.5% conversion) and from checkout start to purchase (77.2% conversion). The overall landing to purchase rate of 17.2% indicates that while the initial traffic is strong, there are potential friction points in the product engagement and the transition to checkout. To optimize this funnel, it would be beneficial to conduct user experience testing on the product pages and streamline the checkout process to reduce any barriers. Additionally, retargeting efforts could focus on re-engaging users who have abandoned their carts to improve overall conversion rates.


In [38]:
# overall summary prompt 

overall_prompt = build_summary_prompt(metrics)

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{"role": "user", "content": overall_prompt}]
)
overall_summary = response.choices[0].message.content


In [39]:
print(overall_summary)

### Overall Campaign Patterns
1. **General Patterns**: Across the campaigns, there is a consistent pattern of significant drop-off at each step of the funnel. Most campaigns exhibit strong landing page views but face considerable challenges in converting those views into purchases. The landing to view rates are generally healthy (ranging from about 64.6% to 65.3%), but subsequent stages experience notable declines. Overall landing to purchase rates hover around 17%, indicating room for improvement in funnel effectiveness.

### Weakest Stages of the Funnel
2. **Weakest Stages**: The most pronounced weaknesses appear at the stages transitioning from:
   - **Product Views to Add to Cart**: This stage shows low retention, with conversion rates often below 50%. Campaigns generally exhibit difficulty converting views into invested actions, suggesting potential engagement issues.
   - **Initial Landing to Product Views**: A significant portion of traffic is also lost at this stage, indicating

Quality check: hallucinations

In [ ]:
def check_alignment(insight, metrics):
    issues = []
    for key, value in metrics.items():
        if isinstance(value, (int,float)) and str(value)[:4] not in insight:
            issues.append(key)
    return issues

metrics['quality_check'] = metrics.apply(lambda row: check_alignment(row['llm_insight'], row.to_dict()), axis = 1)

# text and metric normalisation needed.

In [47]:
metrics

,campaign_name,step1_landing,step2_product_view,step3_add_to_cart,step4_checkout_start,step5_purchase,landing_to_view_rate,view_to_add_rate,add_to_checkout_rate,checkout_to_purchase_rate,landing_to_purchase_rate,llm_insight,quality_check
0,Campaign 1,1548584,1002506,495834,344774,266259,0.647369,0.494595,0.695342,0.772271,0.171937,The data for Campaign 1 reveals a significant ...,"[step1_landing, step2_product_view, step3_add_..."
1,Campaign 10,1768951,1146349,572798,402392,311139,0.648039,0.499672,0.702502,0.773224,0.175889,The conversion rates at each step of Campaign ...,"[step1_landing, step2_product_view, step3_add_..."
2,Campaign 2,1574601,1019944,509995,357840,275353,0.647748,0.500023,0.701654,0.769486,0.174872,The data reveals a significant drop-off across...,"[step1_landing, step2_product_view, step3_add_..."
3,Campaign 3,1567483,1023798,513059,361595,278841,0.653148,0.501133,0.704782,0.771142,0.177891,The campaign shows a significant drop-off at e...,"[step1_landing, step2_product_view, step3_add_..."
4,Campaign 4,1502271,970346,484262,338209,261574,0.645919,0.499061,0.698401,0.773409,0.174119,The traffic data for Campaign 4 reveals a sign...,"[step1_landing, step2_product_view, step3_add_..."
5,Campaign 5,1497988,968031,484843,338912,262432,0.646221,0.500855,0.699014,0.774337,0.175190,The data indicates a significant drop in conve...,"[step1_landing, step2_product_view, step3_add_..."
6,Campaign 6,1640763,1071653,535622,377429,290324,0.653143,0.499809,0.704656,0.769215,0.176945,"Analyzing the funnel for Campaign 6, we observ...","[step1_landing, step2_product_view, step3_add_..."
7,Campaign 7,1621042,1054624,530299,367482,283203,0.650584,0.502832,0.692971,0.770658,0.174704,"Analyzing the funnel from Campaign 7, we obser...","[step1_landing, step2_product_view, step3_add_..."
8,Campaign 8,1662683,1080260,537222,371931,286764,0.649709,0.497308,0.692323,0.771014,0.172471,The data reveals a significant drop-off at eac...,"[step1_landing, step2_product_view, step3_add_..."
9,Campaign 9,1566867,1016160,508767,356161,275265,0.648530,0.500676,0.700047,0.772867,0.175679,The data for Campaign 9 reveals a significant ...,"[step1_landing, step2_product_view, step3_add_..."
